In [1]:
# libraries
import pandas as pd
import os
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import math, time

# local functions
from utils import import_matrix

## Loading data

In [2]:
root    = './'
city    = 'SiouxFalls'

In [3]:
# Importing the networks into a Pandas dataframe consists of a single line of code
# but we can also make sure all headers are lower case and without trailing spaces

net_file = os.path.join(root, city, city + '_net.tntp')
network = pd.read_csv(net_file, skiprows=8, sep='\t')

trimmed_cols = [s.strip().lower() for s in network.columns]
network.columns = trimmed_cols

network.drop(['~', ';'], axis=1, inplace=True)

# rename init_node to from and term_node to to
# check if column has init_node and term_node
if 'init_node' in network.columns:
    network.rename(columns={'init_node': 'from', 'term_node': 'to'}, inplace=True)
    

In [4]:
network

,from,to,capacity,length,free_flow_time,b,power,speed,toll,link_type
0,1,2,25900.200640,6,6,0.15,4,0,0,1
1,1,3,23403.473190,4,4,0.15,4,0,0,1
2,2,1,25900.200640,6,6,0.15,4,0,0,1
3,2,6,4958.180928,5,5,0.15,4,0,0,1
4,3,1,23403.473190,4,4,0.15,4,0,0,1
...,...,...,...,...,...,...,...,...,...,...
71,23,22,5000.000000,4,4,0.15,4,0,0,1
72,23,24,5078.508436,2,2,0.15,4,0,0,1
73,24,13,5091.256152,4,4,0.15,4,0,0,1
74,24,21,4885.357564,3,3,0.15,4,0,0,1


In [5]:
# linkindexordering = list(zip(network['from'], network['to']))
edge_attr = {}
# mlt = False
for row in network.itertuples():
    edge_attr[(row[1], row[2])] = {'capacity':row[3], 'length':row[4], 'FFT':row[5], 'alpha':row[6],
                                    'beta':row[7], 'type':row[10], 'flow':0, 'cost':row[5], 'Oflow':None}
    
edge = [(row[1], row[2]) for row in network.itertuples()]

In [6]:
network_graph = nx.DiGraph()
network_graph.add_edges_from(edge)
nx.set_edge_attributes(network_graph, edge_attr)

In [7]:
origin_destination_file = os.path.join(root, city, city + '_trips.tntp')
matrix, index = import_matrix(origin_destination_file)
# convert matrix dictionary to dataframe
origin_destination_raw = pd.DataFrame.from_dict(matrix, orient='index')
origin_destination_demand = origin_destination_raw.stack().reset_index()
origin_destination_demand.columns = ['Origin', 'Destination', 'demand']

origin_destination_demand = {(row['Origin'], row['Destination']):row['demand'] for index, row in origin_destination_demand.iterrows()}

In [8]:
zones2centroid = {zone: [zone] for zone in origin_destination_raw.columns.to_list()}
#zone2centroid = {zone: [row['osmid'] for index, row in centroids[centroids['zoneID'] == zone].iterrows()] for zone in centroids['zoneID']}

## All or nothing assignment

In [20]:
def AONloading(graph, zone2centroid, demand, compute_sptt=False):
    """
    All-or-Nothing (AON) traffic assignment with shortest path travel time (SPTT) computation.

    Parameters:
        graph (nx.DiGraph): A directed graph representing the road network, with 'cost' as the edge weight.
        zone2centroid (dict): Maps zones to lists of centroid nodes within each zone.
        demand (dict): OD demand values as a dictionary {(origin_zone, destination_zone): demand}.
        compute_sptt (bool): Flag to compute SPTT and EODTT for OD pairs.

    Returns:
        tuple: 
            - SPTT (float): Total shortest path travel time across all OD pairs.
            - x_bar (dict): Edge flow for each edge in the graph.
            - spedges (dict): Shortest paths for each OD pair (optional, if `compute_sptt` is True).
            - EODTT (dict): End-to-end travel times for OD pairs (optional, if `compute_sptt` is True).
    """
    # Initialize outputs
    x_bar = {edge: 0 for edge in graph.edges()}  # Edge flows
    spedges = {}  # Shortest paths for OD pairs
    EODTT = {}  # End-to-end travel times
    SPTT = 0  # Shortest path travel time
    
    # Iterate through origin zones and their centroid nodes
    for origin_zone, origin_nodes in zone2centroid.items():
        # Compute shortest paths from all centroids in the origin zone
        dijkstra_results = [
            nx.single_source_dijkstra(graph, origin_node, weight="cost") for origin_node in origin_nodes
        ]
        
        # Iterate through destination zones and their centroid nodes
        for destination_zone, destination_nodes in zone2centroid.items():
            if origin_zone == destination_zone:
                continue  # Skip intra-zone flows
            
            # Get the demand for the OD pair
            od_demand = demand.get((origin_zone, destination_zone), 0)
            if od_demand <= 0:
                continue  # Skip if no demand
            
            # Find the shortest path among all centroid-to-centroid paths
            shortest_paths = [
                (dijkstra_results[i][0][dest_node], dijkstra_results[i], dest_node)
                for i in range(len(dijkstra_results))
                for dest_node in destination_nodes
                if dest_node in dijkstra_results[i][0]
            ]
            
            if not shortest_paths:
                continue  # Skip if no valid path exists
            
            # Select the shortest path
            shortest_paths.sort(key=lambda x: x[0])  # Sort by travel time
            min_time, dijkstra_result, destination_node = shortest_paths[0]
            path = dijkstra_result[1][destination_node]
            
            # Compute SPTT and store path/EODTT if required
            if compute_sptt:
                SPTT += min_time * od_demand
                spedges[(origin_zone, destination_zone)] = path
                EODTT[(origin_zone, destination_zone)] = min_time
            
            # Update edge flows
            for u, v in zip(path[:-1], path[1:]):
                x_bar[(u, v)] += od_demand
    
    return SPTT, x_bar, spedges, EODTT


In [22]:
G_AON = network_graph.copy()
SPTT, x_bar, spedges, EODTT = AONloading(G_AON, zones2centroid, origin_destination_demand, compute_sptt=True)
print('SPTT: ', SPTT)
print('x_bar: ', x_bar)
print('spedges: ', spedges)
print('EODTT: ', EODTT)

SPTT:  3176000.0
x_bar:  {(1, 2): 3800.0, (1, 3): 6000.0, (2, 1): 3800.0, (2, 6): 6600.0, (3, 1): 6000.0, (3, 4): 10200.0, (3, 12): 6800.0, (6, 2): 6600.0, (6, 5): 10900.0, (6, 8): 16900.0, (4, 3): 10200.0, (4, 5): 13700.0, (4, 11): 7400.0, (12, 3): 6800.0, (12, 11): 11300.0, (12, 13): 12000.0, (5, 4): 14400.0, (5, 6): 10200.0, (5, 9): 7000.0, (11, 4): 6800.0, (11, 10): 19000.0, (11, 12): 11300.0, (11, 14): 17400.0, (9, 5): 7000.0, (9, 8): 800.0, (9, 10): 17000.0, (8, 6): 17600.0, (8, 7): 8100.0, (8, 9): 800.0, (8, 16): 14700.0, (7, 8): 8000.0, (7, 18): 13200.0, (18, 7): 13100.0, (18, 16): 14800.0, (18, 20): 11600.0, (16, 8): 15500.0, (16, 10): 28100.0, (16, 17): 26700.0, (16, 18): 14100.0, (10, 9): 17100.0, (10, 11): 21300.0, (10, 15): 11200.0, (10, 16): 28200.0, (10, 17): 0, (15, 10): 13600.0, (15, 14): 10800.0, (15, 19): 19000.0, (15, 22): 20200.0, (17, 10): 0, (17, 16): 26700.0, (17, 19): 21900.0, (14, 11): 14600.0, (14, 15): 8700.0, (14, 23): 10400.0, (13, 12): 12100.0, (13, 24): 

In [9]:
G_AON = network_graph.copy()
SPTT, x_bar, spedges, EODTT = AONloading(G_AON, zones2centroid, origin_destination_demand, computesptt=True)
print('SPTT: ', SPTT)
print('x_bar: ', x_bar)
print('spedges: ', spedges)
print('EODTT: ', EODTT)

SPTT:  3176000.0
x_bar:  {(1, 2): 3800.0, (1, 3): 6000.0, (2, 1): 3800.0, (2, 6): 6600.0, (3, 1): 6000.0, (3, 4): 10200.0, (3, 12): 6800.0, (6, 2): 6600.0, (6, 5): 10900.0, (6, 8): 16900.0, (4, 3): 10200.0, (4, 5): 13700.0, (4, 11): 7400.0, (12, 3): 6800.0, (12, 11): 11300.0, (12, 13): 12000.0, (5, 4): 14400.0, (5, 6): 10200.0, (5, 9): 7000.0, (11, 4): 6800.0, (11, 10): 19000.0, (11, 12): 11300.0, (11, 14): 17400.0, (9, 5): 7000.0, (9, 8): 800.0, (9, 10): 17000.0, (8, 6): 17600.0, (8, 7): 8100.0, (8, 9): 800.0, (8, 16): 14700.0, (7, 8): 8000.0, (7, 18): 13200.0, (18, 7): 13100.0, (18, 16): 14800.0, (18, 20): 11600.0, (16, 8): 15500.0, (16, 10): 28100.0, (16, 17): 26700.0, (16, 18): 14100.0, (10, 9): 17100.0, (10, 11): 21300.0, (10, 15): 11200.0, (10, 16): 28200.0, (10, 17): 0, (15, 10): 13600.0, (15, 14): 10800.0, (15, 19): 19000.0, (15, 22): 20200.0, (17, 10): 0, (17, 16): 26700.0, (17, 19): 21900.0, (14, 11): 14600.0, (14, 15): 8700.0, (14, 23): 10400.0, (13, 12): 12100.0, (13, 24): 

## Task:
Develop the MSA assignment code that evaulates user equilibrium assignment given the SiouxFalls network and demand



## Work flow to develop MSA
Initial run
* Run dijkstra
* Run AON loading assignment (AON function)
* Update network costs/traveltimes
* Run Dijkstra
* Check relative gap between TSTT and SPTT - Gap is (AON volumes * new costs) / SPTT. SPTT is all demand * min(new costs)

Iterative
* Run AON again
* Calculate MSA volumes/flows
* Update network costs/traveltimes 
* Run dijkstra again
* Check relative gap between TSTT and SPTT - Gap is (MSA volumes * new costs) / SPTT. SPTT is all demand * min(new costs)

In [30]:
def MSA(graph, zone2centroid, demand, max_iterations=100, convergence_threshold=0.05):
    """
    Perform User Equilibrium (UE) Assignment using the Method of Successive Averages (MSA).

    Parameters:
        graph (nx.DiGraph): Directed graph representing the network, with 'cost' as edge weights.
        zone2centroid (dict): Mapping of zones to lists of centroid nodes within each zone.
        demand (dict): OD demand as a dictionary {(origin_zone, destination_zone): demand}.
        max_iterations (int): Maximum number of iterations to run.
        convergence_threshold (float): Threshold for the relative gap to check convergence.

    Returns:
        dict: Final edge flows (volumes) for each edge as {(u, v): flow}.
        list: Convergence history of relative gaps.
    """
    # Initialize edge flows and cost attributes
    for u, v, data in graph.edges(data=True):
        data['flow'] = 0  # Initialize edge flows to zero
        if 'cost' not in data:
            raise ValueError("Graph edges must have an initial 'cost' attribute.")
    
    relative_gaps = []  # To track convergence

    # Step 1: Initial All-or-Nothing (AON) assignment
    print("Running initial AON assignment...")
    SPTT, x_bar, _, _ = AONloading(graph, zone2centroid, demand, compute_sptt=True)

    for iteration in range(1, max_iterations + 1):
        print(f"Iteration {iteration}...")

        # Update flows using MSA: x = x + (1 / iteration) * (x_bar - x)
        for (u, v) in graph.edges():
            graph[u][v]['flow'] = (
                graph[u][v]['flow'] + (1 / iteration) * (x_bar.get((u, v), 0) - graph[u][v]['flow'])
            )

        # Update travel times/costs on the network based on new flows
        update_edge_costs(graph)

        # Step 2: Run shortest path (Dijkstra's algorithm) and AON loading with updated costs
        _, x_bar, _, _ = AONloading(graph, zone2centroid, demand, compute_sptt=False)

        # Step 3: Compute the relative gap
        TSTT = sum(graph[u][v]['flow'] * graph[u][v]['cost'] for u, v in graph.edges())
        SPTT = sum(
            demand.get((origin_zone, destination_zone), 0) * min_cost_between_zones(graph, origin_zone, destination_zone, zone2centroid)
            for origin_zone in zone2centroid
            for destination_zone in zone2centroid
        )
        relative_gap = abs(TSTT - SPTT) / SPTT
        relative_gaps.append(relative_gap)
        
        print(f"Iteration {iteration}: Relative Gap = {relative_gap:.6f}")

        # Check for convergence (relative gap < 0.05)
        if relative_gap < convergence_threshold:
            print("Convergence achieved!")
            break

    # Fixed return statement
    return {(u, v): graph[u][v]['flow'] for u, v in graph.edges()}, relative_gaps



def update_edge_costs(graph):
    """
    Update edge travel times (costs) based on current flows using the BPR (Bureau of Public Roads) function.

    Parameters:
        graph (nx.DiGraph): Directed graph representing the network.

    Notes:
        The BPR function is defined as:
            cost = free_flow_time * (1 + alpha * (flow / capacity)^beta)
    """
    alpha = 0.15
    beta = 4
    for u, v, data in graph.edges(data=True):
        free_flow_time = data.get('free_flow_time', 1)
        capacity = data.get('capacity', 1)
        flow = data['flow']
        data['cost'] = free_flow_time * (1 + alpha * (flow / capacity) ** beta)


def min_cost_between_zones(graph, origin_zone, destination_zone, zone2centroid):
    """
    Compute the minimum cost between two zones using shortest paths.

    Parameters:
        graph (nx.DiGraph): Directed graph representing the network.
        origin_zone (int): Origin zone ID.
        destination_zone (int): Destination zone ID.
        zone2centroid (dict): Mapping of zones to centroid nodes.

    Returns:
        float: Minimum cost between the zones.
    """
    origin_nodes = zone2centroid[origin_zone]
    destination_nodes = zone2centroid[destination_zone]
    
    min_cost = float('inf')
    for origin_node in origin_nodes:
        distances, _ = nx.single_source_dijkstra(graph, origin_node, weight='cost')
        for destination_node in destination_nodes:
            if destination_node in distances:
                min_cost = min(min_cost, distances[destination_node])
    
    return min_cost


In [31]:
final_flows, gaps = MSA(network_graph, zones2centroid, origin_destination_demand)

print("Final Edge Flows:", final_flows)
print("Relative Gaps:", gaps)

Running initial AON assignment...
Iteration 1...
Iteration 1: Relative Gap = 0.825815
Iteration 2...
Iteration 2: Relative Gap = 4.801612
Iteration 3...
Iteration 3: Relative Gap = 0.832032
Iteration 4...
Iteration 4: Relative Gap = 0.244479
Iteration 5...
Iteration 5: Relative Gap = 0.119839
Iteration 6...
Iteration 6: Relative Gap = 0.085773
Iteration 7...
Iteration 7: Relative Gap = 0.075758
Iteration 8...
Iteration 8: Relative Gap = 0.070232
Iteration 9...
Iteration 9: Relative Gap = 0.061142
Iteration 10...
Iteration 10: Relative Gap = 0.051931
Iteration 11...
Iteration 11: Relative Gap = 0.052811
Iteration 12...
Iteration 12: Relative Gap = 0.044665
Convergence achieved!
Final Edge Flows: {(1, 2): 5716.666666666667, (1, 3): 9766.666666666668, (2, 1): 5941.666666666667, (2, 6): 7316.666666666666, (3, 1): 9541.666666666666, (3, 4): 13533.333333333334, (3, 12): 12116.666666666666, (6, 2): 7541.666666666667, (6, 5): 7183.333333333333, (6, 8): 10508.333333333334, (4, 3): 13466.6666666